In [1]:
import requests
import pandas as pd
from datasets import DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
)

c:\Users\Administrator.MMCY\Desktop\10a\EthioMart-W5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Step 1: Download the file
file_id = "1qpzy8eeqlSzkSN4g4yVLAa_ce0ZUovUh"
download_url = f"https://drive.google.com/uc?id={file_id}"
file_path = "dataset.conll"

response = requests.get(download_url)
with open(file_path, "wb") as file:
    file.write(response.content)

In [3]:
# Step 2: Load .conll data
def load_conll_data(file_path):
    sentences, labels, sentence, label = [], [], [], []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                token, entity = line.split()
                sentence.append(token)
                label.append(entity)
            else:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                sentence, label = [], []
        if sentence:
            sentences.append(sentence)
            labels.append(label)
    return sentences, labels

sentences, labels = load_conll_data(file_path)
unique_labels = list(set(label for sublist in labels for label in sublist))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}
numerical_labels = [[label2id[label] for label in sublist] for sublist in labels]

data = {"tokens": sentences, "ner_tags": numerical_labels}
dataset = DatasetDict({"train": Dataset.from_dict(data)})

In [4]:
# Step 3: Tokenize and align labels
model_name = "masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0"  # Change to Masakhane NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_idx is None else label[word_idx] for word_idx in word_ids]
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

c:\Users\Administrator.MMCY\Desktop\10a\EthioMart-W5\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator.MMCY\.cache\huggingface\hub\models--masakhane--afroxlmr-large-ner-masakhaner-1.0_2.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 1340/1340 [00:02<00:00, 606

ValueError: PyTorch needs to be installed to be able to return PyTorch tensors.

In [ ]:

# Step 4: Load model and set up trainer
num_labels = len(unique_labels)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

data_collator = DataCollatorForTokenClassification(tokenizer)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

train_size = int(0.8 * len(tokenized_datasets["train"]))
train_dataset = tokenized_datasets["train"].select(range(train_size))
eval_dataset = tokenized_datasets["train"].select(range(train_size, len(tokenized_datasets["train"])))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

In [ ]:
# Step 5: Train the model
trainer.train()

In [ ]:
# Step 6: Evaluation
metrics = trainer.evaluate(eval_dataset)
print("Evaluation Metrics:", metrics)
